# A deep dive into TOB-WGS' pipeline

Chapter 1: what does `ReblockGVCF` actually do ? Let's examine sample TOB1520 (which is an outlier for some QC metrics)

**version 2.2 of the pipeline: `ReblockGVCF` now supposed NOT to remove low quality variants**

In [ ]:
import hail as hl;

# All datasets in TOB-WGS are using GRCh38
hl.init(default_reference='GRCh38');


Import TOB1520 GVCF and explore structure and content


In [10]:
gvcf = hl.import_vcf('gs://cpg-tob-wgs-test/gvcf/batch1/TOB1520.g.vcf.gz', min_partitions=12, force_bgz=True)
gvcf.describe()
gvcf.info.END.show()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AS_InbreedingCoeff: array<float64>, 
        AS_QD: array<float64>, 
        AS_RAW_BaseQRankSum: str, 
        AS_RAW_MQ: str, 
        AS_RAW_MQRankSum: str, 
        AS_RAW_ReadPosRankSum: str, 
        AS_SB_TABLE: str, 
        BaseQRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQRankSum: float64, 
        RAW_MQandDP: array<int32>, 
        ReadPosRankSum: float64
    }
----------------------------------------
Entry fields:
    'AD': array<int32>
    'DP': int32
    'GQ': int32
    'GT': call


2021-06-19 06:56:17 Hail: INFO: Coerced sorted dataset


,,
locus,alleles,
locus<GRCh38>,array<str>,int32
chr1:10001,"[""T"",""<NON_REF>""]",10016
chr1:10017,"[""C"",""<NON_REF>""]",10026
chr1:10027,"[""A"",""<NON_REF>""]",10027
chr1:10028,"[""C"",""<NON_REF>""]",10038
chr1:10039,"[""A"",""<NON_REF>""]",10041
chr1:10042,"[""C"",""<NON_REF>""]",10044
chr1:10045,"[""A"",""<NON_REF>""]",10045
chr1:10046,"[""C"",""<NON_REF>""]",10046


In [11]:
hl.summarize_variants(gvcf)

2021-06-19 06:57:15 Hail: INFO: Coerced sorted dataset


Number of alleles,Count
2,30075685
3,5202620
4,235909
5,26301
6,6800
7,1784
8,1258
Allele type,Count
Symbolic,35550357
SNP,4464040


Worth noting:

of 35,550,357 variants, 30,075,685 are homozygous reference blocks

Let's load the GVCF after `ReblockGVCF` had been performed

In [13]:
rb_gvcf = hl.import_vcf('gs://cpg-tob-wgs-test-tmp/joint-calling/v2.2/hail/batch/7c4ba7/1/output_gvcf.g.vcf.gz', force_bgz=True, min_partitions=12)
rb_gvcf.describe();
hl.summarize_variants(rb_gvcf);

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_QUALapprox: str, 
        AS_RAW_BaseQRankSum: str, 
        AS_RAW_MQ: str, 
        AS_RAW_MQRankSum: str, 
        AS_RAW_ReadPosRankSum: str, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SB_TABLE: str, 
        AS_SOR: array<float64>, 
        AS_VarDP: str, 
        BaseQRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
   

2021-06-19 07:09:59 Hail: INFO: Coerced sorted dataset


Number of alleles,Count
2,29886034
3,4824254
4,211029
5,23444
6,6243
7,1647
8,1134
Allele type,Count
Symbolic,34953785
SNP,4135327


These are the annotations added by ReblockGVCF

        AC, AF, AN: ???? why would we need that on a single sample ?
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>,
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>,
        AS_QD: array<float64>, 
        AS_QUALapprox: str,
        AS_ReadPosRankSum: array<float64>,
        AS_SOR: array<float64>, 
        AS_VarDP: str,
        FS: float64, 
        MQ: float64,  
        MQ_DP: int32, 
        QD: float64, 
        QUALapprox: int32, 
        RAW_GT_COUNT: array<int32>, 
        SOR: float64, 
        VarDP: int32
and the annotations removed by ReblockGVCF

        MLEAC: array<int32>, 
        MLEAF: array<float64>, 

Additionally we lost more than 5e6 variants: not only homref blocks but also SNPs, etc.
Let's have a look at what we lost:

In [14]:
lost = gvcf.anti_join_rows(rb_gvcf.rows())
hl.summarize_variants(lost)

2021-06-19 07:15:38 Hail: INFO: Coerced sorted dataset
2021-06-19 07:16:36 Hail: INFO: Coerced sorted dataset


Number of alleles,Count
2,280423
3,378366
4,24880
5,2857
6,557
7,137
8,124
Allele type,Count
Symbolic,687344
SNP,328713


We lost mostly homref blocks, as expected, but also other kind of variants, such as 425,204 SNPs

Let's inspect some of these lost variants (more than 2 alleles means it is not a homref block)

In [15]:
lost.filter_rows(hl.len(lost.alleles) == 3).show(50,1,truncate=30)

2021-06-19 07:19:09 Hail: INFO: Coerced sorted dataset
2021-06-19 07:20:05 Hail: INFO: Coerced sorted dataset
2021-06-19 07:21:03 Hail: INFO: Coerced sorted dataset


+---------------+--------------------------------+--------------+--------------+
| locus         | alleles                        | 'TOB1520'.AD | 'TOB1520'.DP |
+---------------+--------------------------------+--------------+--------------+
| locus<GRCh38> | array<str>                     | array<int32> |        int32 |
+---------------+--------------------------------+--------------+--------------+
| chr1:10109    | ["AACCCT","A","<NON_REF>"]     | [11,1,0]     |           12 |
| chr1:10231    | ["CCCCTAACCCTAACCCTAAACCCTA... | [8,0,0]      |            8 |
| chr1:10441    | ["CCCTA","C","<NON_REF>"]      | [11,0,0]     |           11 |
| chr1:16571    | ["G","A","<NON_REF>"]          | [33,2,0]     |           35 |
| chr1:16688    | ["G","A","<NON_REF>"]          | [20,2,0]     |           22 |
| chr1:19226    | ["T","A","<NON_REF>"]          | [27,2,0]     |           29 |
| chr1:49291    | ["C","T","<NON_REF>"]          | [11,1,0]     |           12 |
| chr1:58188    | ["G","T","<NON_REF>"]          | [8,1,0]      |            9 |
| chr1:83965    | ["AAAG","A","<NON_REF>"]       | [4,0,0]      |            4 |
| chr1:99066    | ["TTTC","T","<NON_REF>"]       | [10,2,0]     |           12 |
| chr1:99088    | ["C","G","<NON_REF>"]          | [8,1,0]      |            9 |
| chr1:109262   | ["C","A","<NON_REF>"]          | [24,1,0]     |           25 |
| chr1:183662   | ["G","C","<NON_REF>"]          | [126,13,0]   |          139 |
| chr1:186765   | ["T","C","<NON_REF>"]          | [33,3,0]     |           36 |
| chr1:187204   | ["G","A","<NON_REF>"]          | [31,2,0]     |           33 |
| chr1:189955   | ["A","G","<NON_REF>"]          | [31,2,0]     |           33 |
| chr1:190667   | ["G","A","<NON_REF>"]          | [66,6,0]     |           72 |
| chr1:191859   | ["C","T","<NON_REF>"]          | [64,6,0]     |           70 |
| chr1:197628   | ["G","C","<NON_REF>"]          | [17,2,0]     |           19 |
| chr1:198861   | ["G","C","<NON_REF>"]          | [36,3,0]     |           39 |
| chr1:198872   | ["C","T","<NON_REF>"]          | [34,3,0]     |           37 |
| chr1:261438   | ["A","G","<NON_REF>"]          | [33,3,0]     |           36 |
| chr1:269963   | ["T","A","<NON_REF>"]          | [11,1,0]     |           12 |
| chr1:272389   | ["C","T","<NON_REF>"]          | [41,5,0]     |           46 |
| chr1:275513   | ["A","G","<NON_REF>"]          | [34,2,0]     |           36 |
| chr1:277901   | ["C","A","<NON_REF>"]          | [27,1,0]     |           28 |
| chr1:284892   | ["G","A","<NON_REF>"]          | [37,4,0]     |           41 |
| chr1:285416   | ["T","G","<NON_REF>"]          | [24,2,0]     |           26 |
| chr1:286145   | ["C","CAT","<NON_REF>"]        | [28,2,0]     |           30 |
| chr1:455086   | ["C","A","<NON_REF>"]          | [15,2,0]     |           17 |
| chr1:590492   | ["TA","T","<NON_REF>"]         | [8,0,0]      |            8 |
| chr1:590500   | ["T","C","<NON_REF>"]          | [8,0,0]      |            8 |
| chr1:601557   | ["T","G","<NON_REF>"]          | [17,1,0]     |           18 |
| chr1:601846   | ["T","G","<NON_REF>"]          | [12,2,0]     |           14 |
| chr1:602196   | ["TCCTCTCCTTTCTTCTCTCCATCCC... | [8,0,0]      |            8 |
| chr1:667265   | ["C","T","<NON_REF>"]          | [19,2,0]     |           21 |
| chr1:690012   | ["T","C","<NON_REF>"]          | [20,2,0]     |           22 |
| chr1:712244   | ["G","T","<NON_REF>"]          | [9,1,0]      |           10 |
| chr1:737292   | ["C","T","<NON_REF>"]          | [36,2,0]     |           38 |
| chr1:737436   | ["A","G","<NON_REF>"]          | [41,3,0]     |           44 |
| chr1:757165   | ["A","T","<NON_REF>"]          | [8,3,0]      |           11 |
| chr1:769039   | ["A","ACCCCCCCCCCCCCCCCCCCC... | [9,0,0]      |            9 |
| chr1:780612   | ["TA","T","<NON_REF>"]         | [16,0,0]     |           16 |
| chr1:789646   | ["T","G","<NON_REF>"]          | [21,2,0]     |           23 |
| chr1:789804   | ["G","T","<NO

In [16]:
lost = lost.annotate_rows(nonhomref=hl.agg.count_where(lost.GT.is_non_ref())>0)
lost.aggregate_rows(hl.agg.count_where(lost.nonhomref))

2021-06-19 07:22:01 Hail: INFO: Coerced sorted dataset
2021-06-19 07:22:59 Hail: INFO: Coerced sorted dataset


0

**GOOD, no non HomRef variants were removed**